### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hypertension/GSE181339'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True

# Check for 'Hypertension', 'age', and 'gender' variables
trait_row = 1  # Inferring 'Hypertension' might be linked to metabolic changes in MONW group
age_row = 2  # Unique values found indicating age data availability
gender_row = 0  # Unique values found indicating gender data availability

# Define data conversion functions
def convert_trait(value):
    """Convert Hypertension value to binary type: presence=1, absence=0."""
    if 'MONW' in value:
        return 1  # Inferring hypertension presence in MONW group
    return 0  # Assuming absence for other groups

def convert_age(value):
    """Convert age value to continuous integer type."""
    try:
        age_value = int(value.split(':')[1].strip())
        return age_value
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary type: female=0, male=1."""
    gender_str = value.split(':')[1].strip().lower()
    if gender_str == 'man':
        return 1
    elif gender_str == 'woman':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE181339', './preprocessed/Hypertension/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is not None, run the clinical feature extraction step
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hypertension', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hypertension/trait_data/GSE181339.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# Substep1: Determine the appropriate keys from gene annotation

# The key that maps identifiers in STEP3 is 'NAME'
# The key that stores gene symbols is 'GENE_SYMBOL'
identifier_key = 'NAME'
gene_symbol_key = 'GENE_SYMBOL'

# Substep2: Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# Let’s double check the columns in the mapping to avoid column name issues
print("Columns in gene_mapping_df:", gene_mapping_df.columns)

# Ensure gene_data's index name matches the column name used in mapping
gene_data.index.name = 'NAME'  # Change to match the identifier_key

# Substep3: Apply the mapping to obtain the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hypertension/gene_data/GSE181339.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hypertension')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE181339', './preprocessed/Hypertension/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Hypertension/GSE181339.csv'
    unbiased_merged_data.to_csv(csv_path)
